# This is to read the files and setting the Path

In [ ]:
%%time
from PIL import Image
import glob
import numpy as np
from sklearn.svm import SVC
import os
from sklearn.preprocessing import normalize
from sklearn import svm

#----TRAIN DATA ------
training_path = glob.glob('./training/*.bmp')
#testA_path = glob.glob('./testA/*.bmp')
testA_path = glob.glob('./testA/*.bmp')

np.set_printoptions(threshold=np.nan)

filenames = training_path
filenames1 = testA_path


# **** PREPROCESSING -->

# This is to crop the image to remove Noise (White space)

In [ ]:
def trim(im):
    im = im.convert('L') #convert to grayscale
    w,h = im.size
    i = im.crop((30, 1, w-30, h))
    return i

# This is to open each image, crop it and then resize to 256 again to get equal matrix.

In [ ]:
from resizeimage import resizeimage

img = [trim(Image.open(fn)).resize((256,256)) for fn in filenames]
Xtrain = np.asarray([np.array(im).flatten() for im in img])

img1 = [trim(Image.open(fn)).resize((256,256)) for fn in filenames1]
Xtest = np.asarray([np.array(im).flatten() for im in img1])

print(Xtrain.shape)
print(Xtest.shape)

# Normalization operation on the sets

In [ ]:
from sklearn import preprocessing
Xtrain = preprocessing.normalize(Xtrain, norm='l2')
Xtest = preprocessing.normalize(Xtest, norm='l2')

# XtrainLabels and XtestLabels are label matrix for Train and Test Set's Label's.

In [ ]:
XtrainLabels = []
for i in os.listdir("""/home/bhagyashree/MachineLearning/Assignments/Assignment-4/training"""):
    i = i.split('_')
    i = int(i[0])
    XtrainLabels.append(i)
identiLabels = np.array(XtrainLabels)

XtestLabels = []
for i in os.listdir("""/home/bhagyashree/MachineLearning/Assignments/Assignment-4/testA"""):
    i = i.split('_')
    i = int(i[0])
    XtestLabels.append(i)

# Multi-Class Classification

# MultiClass Classifier using Non-Linear Support Vector Machine - SVC() as a One Vs Rest classifier with RBF kernel

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
clf = svm.SVC(kernel='rbf', C = 10.0, gamma=0.9, decision_function_shape='ovr')
clf.fit(Xtrain, XtrainLabels)
x_predict = clf.predict(Xtest)

print(x_predict)

# Accuracy on the Model

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(XtestLabels, x_predict)
print("accuracy: {0:.2f}%".format((accuracy)*100))